In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import ast     # TO CONVERT STRING INTO LIST
import re
import string
import nltk
import nltk.corpus
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

import pickle

In [2]:
data_credits = pd.read_csv("tmdb_5000_credits.csv")
data_movies = pd.read_csv("tmdb_5000_movies.csv")

# Data Understanding

In [3]:
data_movies.head()

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-07-16,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-03-07,284139100,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124


In [4]:
data_credits.head()

,movie_id,title,cast,crew
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,Spectre,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,49026,The Dark Knight Rises,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,49529,John Carter,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


In [5]:
movie = data_movies.merge(data_credits, on="title")

In [6]:
#movie.to_csv("movie.csv", index=False)

In [7]:
movie.sample(5)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,movie_id,cast,crew
4331,1100000,"[{""id"": 14, ""name"": ""Fantasy""}, {""id"": 878, ""n...",NaN,50037,"[{""id"": 2412, ""name"": ""commune""}, {""id"": 9937,...",en,Beyond the Black Rainbow,"Deep within the mysterious Arboria Institute, ...",7.717237,"[{""name"": ""Chromewood Productions"", ""id"": 28001}]",...,110.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,Beyond Science. Beyond Sanity. Beyond Control.,Beyond the Black Rainbow,5.7,77,50037,"[{""cast_id"": 3, ""character"": ""Barry Nyle"", ""cr...","[{""credit_id"": ""58b04a2a9251411a5800f25d"", ""de..."
1981,24000000,"[{""id"": 35, ""name"": ""Comedy""}, {""id"": 18, ""nam...",NaN,20761,"[{""id"": 6075, ""name"": ""sport""}]",en,Play It to the Bone,"Two aging fighters in LA, friends, get a call ...",5.566203,"[{""name"": ""Touchstone Pictures"", ""id"": 9195}]",...,124.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,NaN,Play It to the Bone,5.7,53,20761,"[{""cast_id"": 1, ""character"": ""Cesar Dominguez""...","[{""credit_id"": ""52fe43f8c3a368484e008853"", ""de..."
3866,8000000,"[{""id"": 99, ""name"": ""Documentary""}, {""id"": 107...",http://empereur.luc-jacquet.com/index_flash.htm,1667,"[{""id"": 970, ""name"": ""parents kids relationshi...",fr,La Marche de l'empereur,"Every year, thousands of Antartica's emperor p...",17.537154,"[{""name"": ""Wild Bunch"", ""id"": 856}, {""name"": ""...",...,80.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,"In the harshest place on Earth, love finds a way.",March of the Penguins,6.9,358,1667,"[{""cast_id"": 10, ""character"": ""Narrator"", ""cre...","[{""credit_id"": ""52fe430ac3a36847f8035fa9"", ""de..."
3651,0,"[{""id"": 18, ""name"": ""Drama""}]",NaN,39269,"[{""id"": 4470, ""name"": ""punk""}, {""id"": 10183, ""...",en,Out of the Blue,Dennis Hopper is a hard-drinking truck driver ...,0.679351,"[{""name"": ""Robson Street"", ""id"": 71953}]",...,94.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,A harrowing drama from the director of Easy Rider,Out of the Blue,6.5,17,10844,"[{""cast_id"": 12, ""character"": ""Nick Harvey"", ""...","[{""credit_id"": ""52fe43c19251416c7501cceb"", ""de..."
2818,15000000,"[{""id"": 35, ""name"": ""Comedy""}, {""id"": 9648, ""n...",NaN,3049,"[{""id"": 630, ""name"": ""dolphin""}, {""id"": 6732, ...",en,Ace Ventura: Pet Detective,He's Ace Ventura: Pet Detective. Jim Carrey is...,41.249004,"[{""name"": ""Warner Bros."", ""id"": 6194}, {""name""...",...,86.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"He's the best there is! (Actually, he's the on...",Ace Ventura: Pet Detective,6.4,1648,3049,"[{""cast_id"": 3, ""character"": ""Ace Ventura"", ""c...","[{""credit_id"": ""562ec46a925141285700b853"", ""de..."


In [8]:
movie.shape

(4809, 23)

In [9]:
movie.describe()

,budget,id,popularity,revenue,runtime,vote_average,vote_count,movie_id
count,4.809000e+03,4809.000000,4809.000000,4.809000e+03,4807.000000,4809.000000,4809.000000,4809.000000
mean,2.902780e+07,57120.571429,21.491664,8.227511e+07,106.882255,6.092514,690.331670,57120.571429
std,4.070473e+07,88653.369849,31.803366,1.628379e+08,22.602535,1.193989,1234.187111,88653.369849
min,0.000000e+00,5.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,5.000000
25%,7.800000e+05,9012.000000,4.667230,0.000000e+00,94.000000,5.600000,54.000000,9012.000000
50%,1.500000e+07,14624.000000,12.921594,1.917000e+07,103.000000,6.200000,235.000000,14624.000000
75%,4.000000e+07,58595.000000,28.350529,9.291317e+07,118.000000,6.800000,737.000000,58595.000000
max,3.800000e+08,459488.000000,875.581305,2.787965e+09,338.000000,10.000000,13752.000000,459488.000000


In [10]:
movie.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4809 entries, 0 to 4808
Data columns (total 23 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   budget                4809 non-null   int64  
 1   genres                4809 non-null   object 
 2   homepage              1713 non-null   object 
 3   id                    4809 non-null   int64  
 4   keywords              4809 non-null   object 
 5   original_language     4809 non-null   object 
 6   original_title        4809 non-null   object 
 7   overview              4806 non-null   object 
 8   popularity            4809 non-null   float64
 9   production_companies  4809 non-null   object 
 10  production_countries  4809 non-null   object 
 11  release_date          4808 non-null   object 
 12  revenue               4809 non-null   int64  
 13  runtime               4807 non-null   float64
 14  spoken_languages      4809 non-null   object 
 15  status               

In [11]:
movie.duplicated().sum()

np.int64(0)

# Data Cleaning

In [12]:
movie = movie[["movie_id", "title", "overview", "genres", "keywords", "cast", "crew"]]

In [13]:
movie.shape

(4809, 7)

In [14]:
movie.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4809 entries, 0 to 4808
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   movie_id  4809 non-null   int64 
 1   title     4809 non-null   object
 2   overview  4806 non-null   object
 3   genres    4809 non-null   object
 4   keywords  4809 non-null   object
 5   cast      4809 non-null   object
 6   crew      4809 non-null   object
dtypes: int64(1), object(6)
memory usage: 263.1+ KB


In [15]:
movie.isnull().mean()*100

movie_id    0.000000
title       0.000000
overview    0.062383
genres      0.000000
keywords    0.000000
cast        0.000000
crew        0.000000
dtype: float64

In [16]:
movie.dropna(inplace=True)

In [17]:
movie.duplicated().sum()

np.int64(0)

# Text Preprocessing

In [18]:
movie.head()

,movie_id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...","[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...","[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...","[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,Spectre,A cryptic message from Bond’s past sends him o...,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...","[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,49026,The Dark Knight Rises,Following the death of District Attorney Harve...,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...","[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...","[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,49529,John Carter,"John Carter is a war-weary, former military ca...","[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 818, ""name"": ""based on novel""}, {""id"":...","[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


In [19]:
movie.iloc[0].genres

'[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]'

In [20]:
# TO CONVERT DICTIONARY'S  VALUES INTO REQUIRED LIST
def convert(object):
    l = []
    for i in ast.literal_eval(object):
        l.append(i["name"])
    return l

movie["genres"] = movie["genres"].apply(convert)

In [21]:
movie.iloc[0].genres

['Action', 'Adventure', 'Fantasy', 'Science Fiction']

In [22]:
movie.iloc[0].keywords

'[{"id": 1463, "name": "culture clash"}, {"id": 2964, "name": "future"}, {"id": 3386, "name": "space war"}, {"id": 3388, "name": "space colony"}, {"id": 3679, "name": "society"}, {"id": 3801, "name": "space travel"}, {"id": 9685, "name": "futuristic"}, {"id": 9840, "name": "romance"}, {"id": 9882, "name": "space"}, {"id": 9951, "name": "alien"}, {"id": 10148, "name": "tribe"}, {"id": 10158, "name": "alien planet"}, {"id": 10987, "name": "cgi"}, {"id": 11399, "name": "marine"}, {"id": 13065, "name": "soldier"}, {"id": 14643, "name": "battle"}, {"id": 14720, "name": "love affair"}, {"id": 165431, "name": "anti war"}, {"id": 193554, "name": "power relations"}, {"id": 206690, "name": "mind and soul"}, {"id": 209714, "name": "3d"}]'

In [23]:
movie["keywords"] = movie["keywords"].apply(convert)

In [24]:
movie.iloc[0].keywords

['culture clash',
 'future',
 'space war',
 'space colony',
 'society',
 'space travel',
 'futuristic',
 'romance',
 'space',
 'alien',
 'tribe',
 'alien planet',
 'cgi',
 'marine',
 'soldier',
 'battle',
 'love affair',
 'anti war',
 'power relations',
 'mind and soul',
 '3d']

In [25]:
# movie.iloc[0].cast

In [26]:
def convert2(object):
    l = []
    x = 0
    for i in ast.literal_eval(object):
        if x !=3:
            l.append(i["name"])
            x = x+1
        else:
            break
    return l

movie["cast"] = movie["cast"].apply(convert2)

In [27]:
movie.iloc[0].cast

['Sam Worthington', 'Zoe Saldana', 'Sigourney Weaver']

In [28]:
#movie["crew"][0]
# movie.iloc[0].crew

In [29]:
def find_director(object):
    l = []
    for i in ast.literal_eval(object):
        if i["job"] == "Director":
             l.append(i["name"])
             break
    return l

movie["crew"] = movie["crew"].apply(find_director)

In [30]:
movie.iloc[0].crew

['James Cameron']

In [31]:
movie.iloc[0].overview

'In the 22nd century, a paraplegic Marine is dispatched to the moon Pandora on a unique mission, but becomes torn between following orders and protecting an alien civilization.'

In [32]:
movie["overview"] = movie["overview"].apply(lambda i: i.split())

In [33]:
movie.iloc[0].overview

['In',
 'the',
 '22nd',
 'century,',
 'a',
 'paraplegic',
 'Marine',
 'is',
 'dispatched',
 'to',
 'the',
 'moon',
 'Pandora',
 'on',
 'a',
 'unique',
 'mission,',
 'but',
 'becomes',
 'torn',
 'between',
 'following',
 'orders',
 'and',
 'protecting',
 'an',
 'alien',
 'civilization.']

In [34]:
movie.head()

,movie_id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"[In, the, 22nd, century,, a, paraplegic, Marin...","[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...","[Sam Worthington, Zoe Saldana, Sigourney Weaver]",[James Cameron]
1,285,Pirates of the Caribbean: At World's End,"[Captain, Barbossa,, long, believed, to, be, d...","[Adventure, Fantasy, Action]","[ocean, drug abuse, exotic island, east india ...","[Johnny Depp, Orlando Bloom, Keira Knightley]",[Gore Verbinski]
2,206647,Spectre,"[A, cryptic, message, from, Bond’s, past, send...","[Action, Adventure, Crime]","[spy, based on novel, secret agent, sequel, mi...","[Daniel Craig, Christoph Waltz, Léa Seydoux]",[Sam Mendes]
3,49026,The Dark Knight Rises,"[Following, the, death, of, District, Attorney...","[Action, Crime, Drama, Thriller]","[dc comics, crime fighter, terrorist, secret i...","[Christian Bale, Michael Caine, Gary Oldman]",[Christopher Nolan]
4,49529,John Carter,"[John, Carter, is, a, war-weary,, former, mili...","[Action, Adventure, Science Fiction]","[based on novel, mars, medallion, space travel...","[Taylor Kitsch, Lynn Collins, Samantha Morton]",[Andrew Stanton]


In [35]:
movie.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4806 entries, 0 to 4808
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   movie_id  4806 non-null   int64 
 1   title     4806 non-null   object
 2   overview  4806 non-null   object
 3   genres    4806 non-null   object
 4   keywords  4806 non-null   object
 5   cast      4806 non-null   object
 6   crew      4806 non-null   object
dtypes: int64(1), object(6)
memory usage: 300.4+ KB


In [36]:
# TO REMOVE THE SPACES WITHIN THE WORDS

movie["genres"] = movie["genres"].apply(lambda x: [i.replace(" ", "") for i in x])
movie["keywords"] = movie["keywords"].apply(lambda x: [i.replace(" ", "") for i in x])
movie["cast"] = movie["cast"].apply(lambda x: [i.replace(" ", "") for i in x])
movie["crew"] = movie["crew"].apply(lambda x: [i.replace(" ", "") for i in x])

In [37]:
movie.head()

,movie_id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"[In, the, 22nd, century,, a, paraplegic, Marin...","[Action, Adventure, Fantasy, ScienceFiction]","[cultureclash, future, spacewar, spacecolony, ...","[SamWorthington, ZoeSaldana, SigourneyWeaver]",[JamesCameron]
1,285,Pirates of the Caribbean: At World's End,"[Captain, Barbossa,, long, believed, to, be, d...","[Adventure, Fantasy, Action]","[ocean, drugabuse, exoticisland, eastindiatrad...","[JohnnyDepp, OrlandoBloom, KeiraKnightley]",[GoreVerbinski]
2,206647,Spectre,"[A, cryptic, message, from, Bond’s, past, send...","[Action, Adventure, Crime]","[spy, basedonnovel, secretagent, sequel, mi6, ...","[DanielCraig, ChristophWaltz, LéaSeydoux]",[SamMendes]
3,49026,The Dark Knight Rises,"[Following, the, death, of, District, Attorney...","[Action, Crime, Drama, Thriller]","[dccomics, crimefighter, terrorist, secretiden...","[ChristianBale, MichaelCaine, GaryOldman]",[ChristopherNolan]
4,49529,John Carter,"[John, Carter, is, a, war-weary,, former, mili...","[Action, Adventure, ScienceFiction]","[basedonnovel, mars, medallion, spacetravel, p...","[TaylorKitsch, LynnCollins, SamanthaMorton]",[AndrewStanton]


In [38]:
movie["tags"] = movie["overview"] + movie["genres"] + movie["keywords"] + movie["cast"] + movie["crew"]

In [39]:
movie["tags"][1]

['Captain',
 'Barbossa,',
 'long',
 'believed',
 'to',
 'be',
 'dead,',
 'has',
 'come',
 'back',
 'to',
 'life',
 'and',
 'is',
 'headed',
 'to',
 'the',
 'edge',
 'of',
 'the',
 'Earth',
 'with',
 'Will',
 'Turner',
 'and',
 'Elizabeth',
 'Swann.',
 'But',
 'nothing',
 'is',
 'quite',
 'as',
 'it',
 'seems.',
 'Adventure',
 'Fantasy',
 'Action',
 'ocean',
 'drugabuse',
 'exoticisland',
 'eastindiatradingcompany',
 "loveofone'slife",
 'traitor',
 'shipwreck',
 'strongwoman',
 'ship',
 'alliance',
 'calypso',
 'afterlife',
 'fighter',
 'pirate',
 'swashbuckler',
 'aftercreditsstinger',
 'JohnnyDepp',
 'OrlandoBloom',
 'KeiraKnightley',
 'GoreVerbinski']

In [40]:
movie.head()

,movie_id,title,overview,genres,keywords,cast,crew,tags
0,19995,Avatar,"[In, the, 22nd, century,, a, paraplegic, Marin...","[Action, Adventure, Fantasy, ScienceFiction]","[cultureclash, future, spacewar, spacecolony, ...","[SamWorthington, ZoeSaldana, SigourneyWeaver]",[JamesCameron],"[In, the, 22nd, century,, a, paraplegic, Marin..."
1,285,Pirates of the Caribbean: At World's End,"[Captain, Barbossa,, long, believed, to, be, d...","[Adventure, Fantasy, Action]","[ocean, drugabuse, exoticisland, eastindiatrad...","[JohnnyDepp, OrlandoBloom, KeiraKnightley]",[GoreVerbinski],"[Captain, Barbossa,, long, believed, to, be, d..."
2,206647,Spectre,"[A, cryptic, message, from, Bond’s, past, send...","[Action, Adventure, Crime]","[spy, basedonnovel, secretagent, sequel, mi6, ...","[DanielCraig, ChristophWaltz, LéaSeydoux]",[SamMendes],"[A, cryptic, message, from, Bond’s, past, send..."
3,49026,The Dark Knight Rises,"[Following, the, death, of, District, Attorney...","[Action, Crime, Drama, Thriller]","[dccomics, crimefighter, terrorist, secretiden...","[ChristianBale, MichaelCaine, GaryOldman]",[ChristopherNolan],"[Following, the, death, of, District, Attorney..."
4,49529,John Carter,"[John, Carter, is, a, war-weary,, former, mili...","[Action, Adventure, ScienceFiction]","[basedonnovel, mars, medallion, spacetravel, p...","[TaylorKitsch, LynnCollins, SamanthaMorton]",[AndrewStanton],"[John, Carter, is, a, war-weary,, former, mili..."


In [41]:
movie_data = movie[["movie_id", "title", "tags"]]
movie_data.head()

,movie_id,title,tags
0,19995,Avatar,"[In, the, 22nd, century,, a, paraplegic, Marin..."
1,285,Pirates of the Caribbean: At World's End,"[Captain, Barbossa,, long, believed, to, be, d..."
2,206647,Spectre,"[A, cryptic, message, from, Bond’s, past, send..."
3,49026,The Dark Knight Rises,"[Following, the, death, of, District, Attorney..."
4,49529,John Carter,"[John, Carter, is, a, war-weary,, former, mili..."


In [42]:
# CONVERTING LIST TO STRING & CONVERTING TO LOWERCASE

movie_data["tags"] = movie_data["tags"].apply(lambda  i: " ".join(i))
movie_data["tags"] = movie_data["tags"].apply(lambda  i: i.lower())

/tmp/ipykernel_27519/1583622536.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movie_data["tags"] = movie_data["tags"].apply(lambda  i: " ".join(i))
/tmp/ipykernel_27519/1583622536.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movie_data["tags"] = movie_data["tags"].apply(lambda  i: i.lower())


In [43]:
movie_data["tags"][0]

'in the 22nd century, a paraplegic marine is dispatched to the moon pandora on a unique mission, but becomes torn between following orders and protecting an alien civilization. action adventure fantasy sciencefiction cultureclash future spacewar spacecolony society spacetravel futuristic romance space alien tribe alienplanet cgi marine soldier battle loveaffair antiwar powerrelations mindandsoul 3d samworthington zoesaldana sigourneyweaver jamescameron'

In [44]:
movie_data.head()

,movie_id,title,tags
0,19995,Avatar,"in the 22nd century, a paraplegic marine is di..."
1,285,Pirates of the Caribbean: At World's End,"captain barbossa, long believed to be dead, ha..."
2,206647,Spectre,a cryptic message from bond’s past sends him o...
3,49026,The Dark Knight Rises,following the death of district attorney harve...
4,49529,John Carter,"john carter is a war-weary, former military ca..."


In [45]:
#REMOVING LINKS, TOKENISATION AND REMOVING PUNCTUATIONS

def preprocessing1(text):
    te = re.sub(r'http\S+', "", text)
    tokens = word_tokenize(te)
    
    punctuations = string.punctuation
    txt = []
    for i in tokens:
        if i not in punctuations:
            txt.append(i)
    return txt

#REMOVING STOPWORDS & CONVERTING TO LOWERCASE

def preprocessing2(text):
    stopword = stopwords.words("english")
    ntxt = []
    for j in text:
        if j not in stopword:
            ntxt.append(j.lower())
    return ntxt

#STEMMING

def preprocessing3(text):
    stem = []
    Stemmer = PorterStemmer()
    for i in text:
        stem.append(Stemmer.stem(i))
    return " ".join(stem)

In [46]:
movie_data["tags"] = movie_data["tags"].apply(preprocessing1)
movie_data["tags"] = movie_data["tags"].apply(preprocessing2)
movie_data["tags"] = movie_data["tags"].apply(preprocessing3)

/tmp/ipykernel_27519/3516406077.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movie_data["tags"] = movie_data["tags"].apply(preprocessing1)
/tmp/ipykernel_27519/3516406077.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movie_data["tags"] = movie_data["tags"].apply(preprocessing2)
/tmp/ipykernel_27519/3516406077.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentat

In [47]:
movie_data["tags"][0]

'22nd centuri parapleg marin dispatch moon pandora uniqu mission becom torn follow order protect alien civil action adventur fantasi sciencefict cultureclash futur spacewar spacecoloni societi spacetravel futurist romanc space alien tribe alienplanet cgi marin soldier battl loveaffair antiwar powerrel mindandsoul 3d samworthington zoesaldana sigourneyweav jamescameron'

# Model Building

In [59]:
from sentence_transformers import SentenceTransformer
from langchain.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.docstore.document import Document

# !pip install langchain chromadb sentence-transformers pandas scikit-learn
# !pip install -U langchain-community
# !pip install -U langchain langchain-community langchainhub

In [49]:
movie_data.head()

,movie_id,title,tags
0,19995,Avatar,22nd centuri parapleg marin dispatch moon pand...
1,285,Pirates of the Caribbean: At World's End,captain barbossa long believ dead come back li...
2,206647,Spectre,cryptic messag bond ’ past send trail uncov si...
3,49026,The Dark Knight Rises,follow death district attorney harvey dent bat...
4,49529,John Carter,john carter war-weari former militari captain ...


In [50]:
movie_data.to_csv("movie_data_preprocessed.csv", index=False)

In [51]:
# Load cleaned and preprocessed movie data
movie_data = pd.read_csv("movie_data_preprocessed.csv")

In [52]:
# Load BERT-based sentence transformer
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")

# Create embeddings from tags
movie_data["embedding"] = movie_data["tags"].apply(lambda x: embedding_model.encode(x, show_progress_bar=False))

In [53]:
movie_data.head()

,movie_id,title,tags,embedding
0,19995,Avatar,22nd centuri parapleg marin dispatch moon pand...,"[-0.016536755, 0.046981633, 0.0120490035, -0.0..."
1,285,Pirates of the Caribbean: At World's End,captain barbossa long believ dead come back li...,"[-0.05846973, -0.08504583, -0.0024445602, -0.0..."
2,206647,Spectre,cryptic messag bond ’ past send trail uncov si...,"[-0.14231715, 0.019504515, -0.04196843, -0.036..."
3,49026,The Dark Knight Rises,follow death district attorney harvey dent bat...,"[-0.078670435, 0.014938344, -0.08796622, -0.05..."
4,49529,John Carter,john carter war-weari former militari captain ...,"[-0.07328494, -0.0059327525, -0.007646511, -0...."


In [54]:
# Convert to LangChain documents
docs = [
    Document(page_content=row["tags"], metadata={"title": row["title"]})
    for _, row in movie_data.iterrows()
]

# Create Chroma vector store
embedding_fn = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
vector_store = Chroma.from_documents(documents=docs, embedding=embedding_fn, persist_directory="./chroma_movies")

# Save for future reuse
vector_store.persist()

/tmp/ipykernel_27519/3736839721.py:8: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_fn = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
/tmp/ipykernel_27519/3736839721.py:12: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vector_store.persist()


In [55]:
def recommend(movie_title: str, k=5):
    # Find the row with this title
    movie_row = movie_data[movie_data["title"] == movie_title]
    if movie_row.empty:
        print("Movie not found.")
        return
    
    query = movie_row.iloc[0]["tags"]

    # Reload vector DB (optional if already in memory)
    vector_store = Chroma(persist_directory="./chroma_movies", embedding_function=embedding_fn)

    # Query the vector DB
    results = vector_store.similarity_search(query, k=k+1)  # +1 to skip the movie itself
    print(f"Top {k} movies similar to '{movie_title}':")
    for doc in results:
        if doc.metadata["title"] != movie_title:
            print(doc.metadata["title"])

In [56]:
recommend("X-Men: Apocalypse")

Top 5 movies similar to 'X-Men: Apocalypse':
X-Men: First Class
X-Men
X-Men: Days of Future Past
X-Men: The Last Stand
Superman IV: The Quest for Peace


/tmp/ipykernel_27519/926943524.py:11: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vector_store = Chroma(persist_directory="./chroma_movies", embedding_function=embedding_fn)


In [57]:
movie_data["title"].sample(10)

2372                 Mongol: The Rise of Genghis Khan
302     Legend of the Guardians: The Owls of Ga'Hoole
3065                                        Bad Words
4589                            What Happens in Vegas
1005                             The Boat That Rocked
1865                               Million Dollar Arm
361                     You Don't Mess with the Zohan
387                                     Air Force One
2484                        My Stepmother is an Alien
2546                           Not Another Teen Movie
Name: title, dtype: object

In [58]:
# cv = CountVectorizer(max_features=5000, stop_words="english")
# tfd = TfidfVectorizer(max_features=3000)

# vector = cv.fit_transform(movie_data["tags"]).toarray()

# # FINDING THE DISTANCES BETWEEN THE MOVIES

# similarity = cosine_similarity(vector)

# print(sorted(list(enumerate(similarity[0])), reverse=True, key=lambda x: x[1])[1:6])

# def recommend(movie):
#     movie_index = movie_data[movie_data["title"] == movie].index[0]     # TO GET THE INDEX VALUE OF THE SEARCHED MOVIE
#     distance = similarity[movie_index]
#     movie_list = sorted(list(enumerate(distance)), reverse=True, key=lambda x: x[1])[1:6]     # TO GET THE FIRST 5 MOVIES SIMILAR TO SEARCHED ONE
    
#     for i in movie_list:
#          print(movie_data.iloc[i[0]].title)

# recommend("X-Men: Apocalypse")

# pickle.dump(movie_data, open("movie_data.pkl","wb"))
# pickle.dump(similarity, open("movie_similarity.pkl", "wb"))